In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders 
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, EncoderDecoderAttn, DecoderAttnRNN
from train_eval import count_parameters, summarize_results, plot_single_learning_curve, load_experiment_log
from train_eval import train_and_eval
import importlib
import pickle as pkl 
import torch

In [2]:
# model identification
MODEL_NAME = 'zh-seq2seq-rnn-attn'
SRC_LANG = 'zh'
TARG_LANG = 'en'

# data processing params  
SRC_MAX_SENTENCE_LEN = 20
TARG_MAX_SENTENCE_LEN = 20
SRC_VOCAB_SIZE = 30000 #30000
TARG_VOCAB_SIZE = 30000 #30000

# model architecture params 
RNN_CELL_TYPE = 'gru'
NUM_LAYERS = 2 #2 
ENC_HIDDEN_DIM = 256 #512
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM #2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0.2 # to actually implement
DEC_DROPOUT = 0.2 # to actually implement
USE_ATTN = False

# training params  
BATCH_SIZE = 64 #32
NUM_EPOCHS = 20
LR = 0.0003 # 0.0005
OPTIMIZER = 'Adam'
LAZY_TRAIN = False

In [3]:
# store as dict to save to results later 
params = {'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 'rnn_cell_type': RNN_CELL_TYPE, 
          'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 
          'src_vocab_size': SRC_VOCAB_SIZE, 'targ_vocab_size': TARG_VOCAB_SIZE, 
          'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 'dec_hidden_dim': DEC_HIDDEN_DIM,
          'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 'clip_grad_max_norm': CLIP_GRAD_MAX_NORM,
          'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 'use_attn': USE_ATTN, 
          'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 'learning_rate': LR, 'optimizer': OPTIMIZER, 
          'lazy_train': LAZY_TRAIN} 

In [4]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [5]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab)
data_minibatch = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=BATCH_SIZE) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=1000)

In [6]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [7]:
# define model 

encoder = EncoderRNN(rnn_cell_type=RNN_CELL_TYPE, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     src_max_sentence_len=SRC_MAX_SENTENCE_LEN, enc_dropout=ENC_DROPOUT, 
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))

# without attention 
# decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS,
#                      targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                      pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id'])

# with attention 
decoder = DecoderAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
                         num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
                         targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, dec_dropout=DEC_DROPOUT, 
                         pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [ ]:
model, results = train_and_eval(
    model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
    params=params, vocab=vocab, print_intermediate=100, save_checkpoint=True, save_to_log=True, 
    lazy_eval=True, print_attn=True, inspect_samples=1)

Epoch: 0.00, Train Loss: 0.00, Val Loss: 10.23, Train BLEU: 0.00, Val BLEU: 0.07, Minutes Elapsed: 0.17
Sampling from val predictions...
Source: 超过 70 的 家庭 家庭暴力 暴力 谋杀 发生 生在 受害 受害者 结束 这段 关系 后 在 她 离开 之后 因为
Reference: over 70 percent of domestic violence murders happen after the victim has ended the relationship , after she &apos;s
Model: <SOS> soaring soaring soaring the the sleeper <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[0.0409, 0.0424, 0.0447, 0.0474, 0.0460, 0.0486, 0.0511, 0.0518, 0.0488,
         0.0495, 0.0511, 0.0521, 0.0505, 0.0494, 0.0515, 0.0537, 0.0556, 0.0557,
         0.0550, 0.0544],
        [0.0409, 0.0424, 0.0447, 0.0474, 0.0459, 0.0485, 0.0511, 0.0518, 0.0488,
         0.0495, 0.0511, 0.0521, 0.0505, 0.0494, 0.0515, 0.0537, 0.0556, 0.0557,
         0.0550, 0.0544],
        [0.0409, 0.0424, 0.0447, 0.0474, 0.0459, 0.0485, 0.0511, 0.0518, 0.0488,
         0.0495, 0.0510, 0.0521, 0.0505, 0.0494, 0.0515, 0.053

Epoch: 0.16, Train Loss: 0.00, Val Loss: 6.20, Train BLEU: 0.00, Val BLEU: 2.34, Minutes Elapsed: 15.32
Sampling from val predictions...
Source: 我们 是 一个 自由 的 组织 由来 来自 不同 同行 <UNK> 行业 不同 <UNK> 城市 的 <UNK> 组成 大家 全都
Reference: what we do , we &apos;re a <UNK> kind of group , where it &apos;s composed of gardeners from
Model: <SOS> and &apos;s , , , , , , , the the , . , . . . <EOS> .
Attention Weights: tensor([[0.0273, 0.0744, 0.0804, 0.0872, 0.0982, 0.0917, 0.0893, 0.0834, 0.0736,
         0.0551, 0.0095, 0.0618, 0.0473, 0.0074, 0.0478, 0.0354, 0.0038, 0.0190,
         0.0069, 0.0005],
        [0.0373, 0.0652, 0.0718, 0.0765, 0.0811, 0.0799, 0.0792, 0.0766, 0.0715,
         0.0611, 0.0226, 0.0649, 0.0549, 0.0188, 0.0526, 0.0415, 0.0108, 0.0227,
         0.0099, 0.0014],
        [0.0402, 0.0608, 0.0665, 0.0702, 0.0732, 0.0731, 0.0729, 0.0715, 0.0683,
         0.0614, 0.0303, 0.0640, 0.0570, 0.0264, 0.0547, 0.0458, 0.0172, 0.0284,
         0.0150, 0.0031],
        [0.0413, 0.0599, 0.0652, 0.

Epoch: 0.33, Train Loss: 0.00, Val Loss: 6.09, Train BLEU: 0.00, Val BLEU: 1.80, Minutes Elapsed: 30.47
Sampling from val predictions...
Source: 我们 用 的 就是 清洁 <UNK> 用 的 钢丝 <UNK> 钢丝 <UNK> 可以 <UNK> 很小 小块 我们 把 这些 些小
Reference: so what we can do is use steel wool just to clean <UNK> , and the steel wool we
Model: <SOS> and , , a a , , , , , the the . <EOS> <EOS> . . . <EOS>
Attention Weights: tensor([[0.3186, 0.1996, 0.0811, 0.0643, 0.0076, 0.0012, 0.0447, 0.0184, 0.0035,
         0.0005, 0.0025, 0.0005, 0.0083, 0.0006, 0.0138, 0.0017, 0.1391, 0.0703,
         0.0215, 0.0022],
        [0.1303, 0.1314, 0.0921, 0.0866, 0.0243, 0.0071, 0.0808, 0.0480, 0.0162,
         0.0040, 0.0129, 0.0041, 0.0280, 0.0047, 0.0374, 0.0087, 0.1388, 0.0940,
         0.0429, 0.0078],
        [0.1058, 0.1088, 0.0859, 0.0829, 0.0335, 0.0134, 0.0798, 0.0548, 0.0248,
         0.0086, 0.0209, 0.0088, 0.0372, 0.0096, 0.0456, 0.0153, 0.1157, 0.0866,
         0.0485, 0.0136],
        [0.0817, 0.0847, 0.0748, 0.0739, 0.04

Epoch: 0.49, Train Loss: 0.00, Val Loss: 5.89, Train BLEU: 0.00, Val BLEU: 4.00, Minutes Elapsed: 45.58
Sampling from val predictions...
Source: 你 可以 以是 是非 非凡 的 <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: you can be extraordinary . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Model: <SOS> i i &apos;t a . . <EOS> . <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[0.9983, 0.0016, 0.0001, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.9873, 0.0112, 0.0007, 0.0005, 0.0001, 0.0001, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.8697, 0.0973, 0.0163, 0.0093, 0.0032, 0.0028, 0.0013, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0

Epoch: 0.66, Train Loss: 0.00, Val Loss: 5.80, Train BLEU: 0.00, Val BLEU: 3.34, Minutes Elapsed: 60.73
Sampling from val predictions...
Source: 事实 实是 康 纳 在 一个 周五 的 晚上 回到 家 告诉 我 他 辞掉 了 他 的 工作 他
Reference: instead , <UNK> came home one friday evening and he told me that he had quit his job that
Model: <SOS> and , the , i i i , i i , to . . . . . <EOS> .
Attention Weights: tensor([[7.3082e-01, 9.8949e-03, 1.8067e-02, 1.6319e-02, 9.7914e-02, 4.6987e-02,
         3.4414e-03, 1.8736e-02, 1.1331e-03, 1.5048e-03, 5.5812e-04, 3.8547e-03,
         5.0632e-02, 8.3380e-05, 1.2204e-05, 1.9021e-05, 1.5853e-05, 2.0359e-06,
         8.4031e-07, 2.3728e-06],
        [3.6100e-01, 2.2830e-02, 6.7364e-02, 5.7900e-02, 2.1157e-01, 9.9073e-02,
         1.5521e-02, 4.7642e-02, 5.7081e-03, 6.1564e-03, 2.1261e-03, 6.2002e-03,
         9.5707e-02, 6.6784e-04, 1.2931e-04, 1.6424e-04, 1.6261e-04, 3.2094e-05,
         1.3005e-05, 3.2296e-05],
        [6.0216e-02, 3.5948e-02, 9.0614e-02, 8.9853e-02, 1.5982e-01, 1.2

...

Epoch: 5.49, Train Loss: 0.00, Val Loss: 5.15, Train BLEU: 0.00, Val BLEU: 5.70, Minutes Elapsed: 513.24
Sampling from val predictions...
Source: 而 可持续性 持续 持续性 的 搞笑 之处 处在 在于 你 必须 维持 它 <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: the funny thing about sustainability , you have to sustain it . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Model: <SOS> and the of of that , you can to it it . <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[0.0652, 0.2703, 0.4402, 0.1539, 0.0123, 0.0568, 0.0007, 0.0002, 0.0003,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0150, 0.0807, 0.3899, 0.1548, 0.0297, 0.3022, 0.0189, 0.0043, 0.0036,
         0.0008, 0.0001, 0.0000, 0.0000, 0.0002, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0407, 0.0503, 0.2921, 0.2823, 0.0843, 0.1574, 0.0494, 0.0187, 0.0108,
         0.0035, 0.0014, 0.0006, 0.0005, 0.0081, 0.0000, 0.0000, 0.0000, 0.0000,
     

Epoch: 5.66, Train Loss: 0.00, Val Loss: 5.13, Train BLEU: 0.00, Val BLEU: 5.45, Minutes Elapsed: 528.81
Sampling from val predictions...
Source: 她们 带 着 我 走下 一段 <UNK> 狭窄 的 楼梯 到 了 一个 肮脏 昏暗 的 地下 地下室 <EOS> <PAD>
Reference: they ushered me down a narrow set of stairs that led to this dirty , dimly fluorescent lit basement
Model: <SOS> they they me a a a of , a , a a a . . . . <EOS> <EOS>
Attention Weights: tensor([[0.9588, 0.0384, 0.0026, 0.0002, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.2591, 0.4584, 0.1129, 0.1571, 0.0097, 0.0010, 0.0002, 0.0003, 0.0002,
         0.0005, 0.0001, 0.0001, 0.0001, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0002, 0.0000],
        [0.0228, 0.0850, 0.1329, 0.6503, 0.0880, 0.0113, 0.0026, 0.0021, 0.0005,
         0.0023, 0.0003, 0.0005, 0.0005, 0.0001, 0.0000, 0.0000, 0.0001, 0.0002,
         0.0005, 0.0000],
        [0.0376, 0.0121, 0.020

Epoch: 5.77, Train Loss: 0.00, Val Loss: 5.29, Train BLEU: 0.00, Val BLEU: 5.54, Minutes Elapsed: 539.20
Sampling from val predictions...
Source: 图象 象是 强大 的 但 同时 又 是 表面 <UNK> 的 <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: image is powerful , but also image is superficial . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Model: <SOS> the is the , but it &apos;s the <EOS> . <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[0.5010, 0.2078, 0.2707, 0.0037, 0.0114, 0.0050, 0.0001, 0.0000, 0.0000,
         0.0000, 0.0001, 0.0001, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.1179, 0.0639, 0.7049, 0.0138, 0.0728, 0.0231, 0.0005, 0.0002, 0.0000,
         0.0000, 0.0003, 0.0026, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0443, 0.0544, 0.4674, 0.0339, 0.2880, 0.0832, 0.0110, 0.0032, 0.0003,
         0.0006, 0.0025, 0.0112, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

Epoch: 5.93, Train Loss: 0.00, Val Loss: 5.24, Train BLEU: 0.00, Val BLEU: 5.31, Minutes Elapsed: 554.79
Sampling from val predictions...
Source: 见到 他 是 在 一个 收容 收容所 所里 free the <UNK> 组织 用于 <UNK> 奴役 受害 受害者 的 一个 地方
Reference: i met him at a shelter where free the slaves <UNK> victims of slavery . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> she was a a a a of a a a of a the <EOS> . <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[1.1885e-02, 9.7155e-01, 1.2684e-02, 2.4145e-03, 2.6639e-04, 5.8961e-04,
         4.0956e-04, 2.3395e-05, 1.0759e-04, 1.6818e-05, 4.3100e-06, 1.4082e-05,
         1.0797e-06, 1.8564e-07, 9.0339e-07, 6.6034e-06, 4.2451e-06, 5.9254e-06,
         4.7520e-06, 6.4062e-06],
        [8.4084e-03, 6.5557e-01, 2.1906e-01, 9.8239e-02, 7.5921e-03, 6.1119e-03,
         1.6397e-03, 3.2996e-04, 1.3266e-03, 4.0070e-04, 1.8687e-04, 5.9743e-04,
         8.5648e-05, 2.1979e-05, 4.2621e-05, 6.5949e-05, 5.9891e-05, 3.7297e-05,
         1.2278e-04, 9.5450e-05],
        [1.1580e-02, 1.3650e

Epoch: 6.00, Train Loss: 0.00, Val Loss: 5.19, Train BLEU: 0.00, Val BLEU: 5.12, Minutes Elapsed: 561.14
Sampling from val predictions...
Source: 这个 个人 还 得 去 <UNK> 条约 和 接见 <UNK> <UNK> <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: this guy has to go and sign treaties and meet foreign <UNK> . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD>
Model: <SOS> this person is a to with and and the <EOS> . <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[0.3968, 0.5792, 0.0225, 0.0012, 0.0002, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0309, 0.9279, 0.0224, 0.0132, 0.0036, 0.0003, 0.0007, 0.0001, 0.0003,
         0.0001, 0.0001, 0.0002, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0616, 0.4068, 0.2330, 0.1209, 0.1031, 0.0133, 0.0154, 0.0046, 0.0093,
         0.0080, 0.0113, 0.0128, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [ ]:
summarize_results(load_experiment_log())[['dt_created', 'num_epochs', 'learning_rate', 'clip_grad_max_norm', 'val_loss']].head()

In [ ]:
plot_single_learning_curve(results)

In [ ]:
# Epoch: 199.00, Train Loss: 0.32, Val Loss: 13.19, Train BLEU: 98.94, Val BLEU: 0.27
plot_single_learning_curve(results)

In [ ]:
# with attention energies = v_broadcast.bmm(torch.tanh(self.attn(concat)).transpose(1, 2)) # switched order  
# Epoch: 199.00, Train Loss: 0.63, Val Loss: 12.82, Train BLEU: 92.05, Val BLEU: 0.38
plot_single_learning_curve(results)

In [ ]:
for i, token in enumerate(vocab[SRC_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
for i, token in enumerate(vocab[TARG_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
import torch
x = torch.arange(0, 3*5*10).view(3, 5, 10)
print(x)
y = x[1:, :, :]
print(y)
z = y.view(-1, 10)
print(z)

In [ ]:
t = torch.arange(0, 2*5).view(5, 2)
print(t)
u = t.contiguous().view(-1)
print(u)
v = t.permute(1, 0)
print(v)
w = v.contiguous().view(-1)
print(w)

In [ ]:
a = torch.arange(0, 2*1*300)
print(a)
b = a.view(-1, 1, 300)
print(b.size())

In [ ]:
for i, (src_idxs, targ_idxs, src_lens, targ_lens) in enumerate(full_loaders['train']):
#     print(i)
#     print(src_idxs.size())
#     print(src_idxs)
#     print(src_lens)
#     print(targ_idxs.size())
#     print(targ_idxs)
#     print(targ_lens)
    id2token = vocab[SRC_LANG]['id2token']
    test_tensor = src_idxs
    list_of_lists = test_tensor.numpy().astype(int).tolist()
    to_token = lambda l: ' '.join([id2token[idx] for idx in l])
    list_of_lists_tokens = [to_token(l) for l in list_of_lists] 
    break 